In [1]:
from pathlib import Path
import sys
import numpy as np
import xarray as xr
from datetime import datetime 

METHODS_DEV = Path("/home/jovyan/methods_development/mapping/pangeo")  #

if str(METHODS_DEV) not in sys.path:
    sys.path.append(str(METHODS_DEV))

In [2]:
from utils import convenient as cv

In [3]:
gcs = cv.get_gcs(path=None, asynchronous=False)
mapper = gcs.get_mapper("pangeo-argo-eke/data/EasyOneArgoTSLite_v01_Spaghetti")
ds = xr.open_zarr(mapper, consolidated=True)

In [12]:
!pip install gsw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 9.1 MB/s  0:00:00m eta 0:00:01


In [4]:
import gsw

In [5]:
p_ref = 0.0  # reference pressure
start = datetime.now()
latitude = ds.latitude.to_numpy()
longitude = ds.longitude.to_numpy()
potrho = np.full(shape=ds.salinity.shape, fill_value=np.nan, dtype=np.float64)
print("start", start)
for plevel, p in enumerate(ds.pressure):

    p0 = p.item()
    p_vec = np.full_like(longitude, p0, dtype=float)
    
    ds_plevel = ds.isel(pressure=plevel)
    sp = ds_plevel.salinity.to_numpy()
    t  = ds_plevel.temperature.to_numpy()
    SA = gsw.SA_from_SP(sp, p_vec, longitude, latitude)
    potrho[:,plevel] = gsw.pot_rho_t_exact(SA, t, p_vec, p_ref)
    print("    ", datetime.now()-start, "plevel:", plevel, '      min:', np.nanmin(potrho[:,plevel]).item(), 'max:', np.nanmax(potrho[:,plevel]).item())
    
potrho_outfile = 'pangeo-argo-eke/data/potrho_from_EOATS_v01'
new_mapper = gcs.get_mapper(potrho_outfile)
potrho_da = xr.DataArray(
    potrho,
    dims=ds.salinity.dims,      
    coords=ds.salinity.coords, 
    name="potential_density",
    attrs={
        "long_name": "Potential density of seawater",
        "units": "kg m-3",
        "standard_name": "sea_water_potential_density",
        "p_ref": p_ref,
        "TEOS10": "https://teos-10.org/pubs/gsw/html/gsw_pot_rho_t_exact.html",
        "initialized_date": str( datetime.now() ),
        "filename": potrho_outfile, 
    },
)
potrho_da = potrho_da.assign_coords(
    latitude=ds.latitude,
    longitude=ds.longitude,
    time=ds.time,
)
ds_potrho = potrho_da.to_dataset()

ds_potrho.chunk(
    {'profilelocation_index': latitude.size, 'pressure': 1}
).to_zarr(
    new_mapper,
    mode="w",
    consolidated=True,
    compute=True
)

start 2025-12-01 20:16:36.511096
    0:00:07.199709 plevel: 0       min: 1002.278104863427 max: 1030.8849323810991
    0:00:10.466451 plevel: 1       min: 999.8954506779482 max: 1031.9797278359354
    0:00:13.858586 plevel: 2       min: 999.8956674812588 max: 1031.993327525638
    0:00:17.388076 plevel: 3       min: 999.8956761857044 max: 1031.9634935635413
    0:00:20.910251 plevel: 4       min: 999.8956328864598 max: 1031.9602402163587
    0:00:24.528752 plevel: 5       min: 999.895693499059 max: 1031.974686202084
    0:00:28.152339 plevel: 6       min: 998.3151186071685 max: 1032.0055300061588
    0:00:31.691217 plevel: 7       min: 999.8956587308738 max: 1032.0390137819988
    0:00:35.200168 plevel: 8       min: 999.5521106105446 max: 1032.0466997562758
    0:00:38.679348 plevel: 9       min: 999.8955718602762 max: 1032.008765968696
    0:00:42.225543 plevel: 10       min: 999.8957362259407 max: 1032.004158601059
    0:00:45.788320 plevel: 11       min: 999.8957446760511 max: 1032.

/srv/conda/envs/notebook/lib/python3.13/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [6]:
import matplotlib.pyplot as plt

In [19]:
ds_potrho_chunked = xr.open_zarr(gcs.get_mapper(ds_potrho.potential_density.filename), consolidated=True)
ds_potrho_chunked

<xarray.Dataset> Size: 2GB
Dimensions:                (profilelocation_index: 2651963, pressure: 107)
Coordinates:
  * profilelocation_index  (profilelocation_index) int64 21MB 0 1 ... 2651962
  * pressure               (pressure) float64 856B 2.0 5.0 ... 5.8e+03 6e+03
    latitude               (profilelocation_index) float32 11MB dask.array<chunksize=(2651963,), meta=np.ndarray>
    longitude              (profilelocation_index) float32 11MB dask.array<chunksize=(2651963,), meta=np.ndarray>
    time                   (profilelocation_index) datetime64[ns] 21MB dask.array<chunksize=(2651963,), meta=np.ndarray>
Data variables:
    potential_density      (profilelocation_index, pressure) float64 2GB dask.array<chunksize=(2651963, 1), meta=np.ndarray>